In [29]:
import pandas as pd
movies = pd.read_csv('./data/movie_summaries_ada/movie.metadata.tsv', sep='\t', header=None)
cpi_data = pd.read_csv("./data/cpi_data/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_5994714.csv", sep=',', skiprows=4)
budget_data = pd.read_csv('./data/movies_budget/movies_metadata.csv', sep=',')

/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/2582371815.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  budget_data = pd.read_csv('./data/movies_budget/movies_metadata.csv', sep=',')


In [129]:
#Get and clean budget data
budget = budget_data[["original_title", "release_date", "runtime", "budget", "revenue"]]
print(len(budget))
budget["revenue_budget"] = budget["revenue"]
budget = budget.drop(columns=["revenue"])
budget["budget"] = pd.to_numeric(budget["budget"], errors='coerce').dropna().astype(int)
budget['year_release'] = pd.to_datetime(budget["release_date"], infer_datetime_format=True, errors="coerce").dt.year
budget = budget.dropna()
budget = budget.drop(columns=["release_date"])
budget['year_release'] =  budget['year_release'].astype(int)
budget = budget[budget["budget"] != 0]
print(len(budget))
budget.head()

45466
8876


/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/729960586.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget["revenue_budget"] = budget["revenue"]
/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/729960586.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  budget['year_release'] = pd.to_datetime(budget["release_date"], infer_datetime_format=True, errors="coerce").dt.year


,original_title,runtime,budget,revenue_budget,year_release
0,Toy Story,81.0,30000000.0,373554033.0,1995
1,Jumanji,104.0,65000000.0,262797249.0,1995
3,Waiting to Exhale,127.0,16000000.0,81452156.0,1995
5,Heat,170.0,60000000.0,187436818.0,1995
6,Sabrina,127.0,58000000.0,0.0,1995


In [134]:
#get and clean movies data
movies.columns = ['wikipedia_id', 'freebase_id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres']
movie_rev = movies[["name", "release_date" , "runtime", "revenue"]]
movie_rev["revenue_ada"] = movie_rev["revenue"]
movie_rev['year_release'] = pd.to_datetime(movie_rev["release_date"], infer_datetime_format=True, errors="coerce").dt.year
movie_rev = movie_rev.drop(columns=["revenue", "release_date"])
print(len(movie_rev))
movie_rev = movie_rev.dropna()
movie_rev['year_release'] =  movie_rev['year_release'].astype(int)
print(len(movie_rev))
movie_rev.head()


81741
6730


/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/3817896483.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_rev["revenue_ada"] = movie_rev["revenue"]
/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/3817896483.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  movie_rev['year_release'] = pd.to_datetime(movie_rev["release_date"], infer_datetime_format=True, errors="coerce").dt.year
/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/3817896483.py:5: SettingWithCopyWarning: 
A v

,name,runtime,revenue_ada,year_release
0,Ghosts of Mars,98.0,14010832.0,2001
7,Alexander's Ragtime Band,106.0,3600000.0,1938
13,Henry V,137.0,10161099.0,1989
17,Mary Poppins,139.0,102272727.0,1964
21,New Rose Hotel,92.0,21521.0,1999


In [160]:
money_data = budget.merge(movie_rev, left_on=["original_title", "year_release", "runtime"], right_on=["name", "year_release" , "runtime"], how='left')
#drop the useless columns
money_data = money_data.drop(columns=["name"])
print(len(money_data))
money_data = money_data[['original_title', 'runtime', 'year_release', 'budget', 'revenue_budget','revenue_ada']]
money_data.head()

8876


,original_title,runtime,year_release,budget,revenue_budget,revenue_ada
0,Toy Story,81.0,1995,30000000.0,373554033.0,NaN
1,Jumanji,104.0,1995,65000000.0,262797249.0,262797249.0
2,Waiting to Exhale,127.0,1995,16000000.0,81452156.0,NaN
3,Heat,170.0,1995,60000000.0,187436818.0,187436818.0
4,Sabrina,127.0,1995,58000000.0,0.0,53458319.0


In [138]:
money_data['revenue_budget'] = money_data['revenue_budget'].replace(0.0, pd.NA)

# Step 2: Create 'revenue_final' column
money_data['revenue_final'] = money_data['revenue_budget'].fillna(money_data['revenue_ada'])
money_data = money_data.drop(columns=["revenue_budget", "revenue_ada"])

# Step 3: Drop rows where 'revenue_final' is NaN or 0
money_data = money_data.dropna(subset=['revenue_final' , "budget"])
print(len(money_data))
money_data.head()

5567


,original_title,runtime,year_release,budget,revenue_final
0,Toy Story,81.0,1995,30000000.0,373554033.0
1,Jumanji,104.0,1995,65000000.0,262797249.0
2,Waiting to Exhale,127.0,1995,16000000.0,81452156.0
3,Heat,170.0,1995,60000000.0,187436818.0
4,Sabrina,127.0,1995,58000000.0,53458319.0


In [142]:
#Get CPI data
cpi_df = cpi_data[cpi_data["Country Code"] == "USA"]
cpi_df = cpi_df.drop(columns=["Country Name",	"Country Code",	"Indicator Name", "Indicator Code", 'Unnamed: 67'])
cpi_df

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
251,1.457976,1.070724,1.198773,1.239669,1.278912,1.585169,3.015075,2.772786,4.271796,5.462386,...,1.464833,1.622223,0.118627,1.261583,2.13011,2.442583,1.81221,1.233584,4.697859,8.0028


In [149]:
# Step 1: Transform CPI Data
years = [str(year) for year in range(1960, 2023)]  # Adjust range as needed
cpi_long_format = cpi_df.melt(var_name='Year', value_name='InflationRate', value_vars=years)
cpi_long_format['Year'] = cpi_long_format['Year'].astype(int)

# Step 2: Calculate Cumulative Inflation Factors
def calculate_cumulative_inflation(start_year, cpi_data):
    # Sort data to ensure chronological order
    sorted_cpi = cpi_data.sort_values(by='Year')
    inflation_rates = sorted_cpi[sorted_cpi['Year'] >= start_year]['InflationRate'] / 100
    cumulative_inflation = (1 + inflation_rates).cumprod().iloc[-1] - 1
    return cumulative_inflation

money_data['CumulativeInflation'] = money_data['year_release'].apply(lambda x: calculate_cumulative_inflation(x, cpi_long_format))

# Step 3: Adjust Budget and Revenue Values
money_data['budget_adjusted'] = money_data.apply(lambda row: row['budget'] * (1 + row['CumulativeInflation']), axis=1)
money_data['revenue_final_adjusted'] = money_data.apply(lambda row: row['revenue_final'] * (1 + row['CumulativeInflation']), axis=1)

money_data['budget_adjusted'] =  money_data['budget_adjusted'].astype(int)
money_data['revenue_final_adjusted'] =  money_data['revenue_final_adjusted'].astype(int)



# Now df contains inflation-adjusted budget and revenue_final


In [161]:
money_data = money_data.sort_values(by='year_release').reset_index(drop=True)
print(len(money_data))
money_data.head()

8876


,original_title,runtime,year_release,budget,revenue_budget,revenue_ada
0,Le Voyage dans la Lune,14.0,1902,5985.0,0.0,NaN
1,The Great Train Robbery,12.0,1903,150.0,0.0,NaN
2,Voyage à travers l'impossible,20.0,1904,7500.0,0.0,NaN
3,Rescued by Rover,7.0,1905,37.0,0.0,NaN
4,Cabiria,148.0,1914,210.0,0.0,NaN


In [158]:
clean_financial_df = money_data[["original_title",	"runtime",	"year_release",	"budget_adjusted",	"revenue_final_adjusted"]]
clean_financial_df.head()

,original_title,runtime,year_release,budget_adjusted,revenue_final_adjusted
0,The Cheat,59.0,1915,173795,1379092
1,The Birth of a Nation,193.0,1915,1003961,110435817
2,"20,000 Leagues Under the Sea",105.0,1916,2007923,80316958
3,Mickey,93.0,1918,2509904,80316958
4,The Kid,68.0,1921,2509904,25099049


In [159]:
#safe to csv
#clean_financial_df.to_csv('./cleaned_datasets/financial_data.csv', index=False)